In [1]:
import json
import pickle
import numpy as np
import random

In [2]:
with open('/Users/navjot/projects/capstone/Radiology-Report-Genration-using-MultiModal-PrototypeNetwork/files/iu_xray/labels_14.pickle','rb') as myfile:
    labels = pickle.load(myfile)

with open('/Users/navjot/projects/capstone/Radiology-Report-Genration-using-MultiModal-PrototypeNetwork/data/iu_xray/annotation.json') as json_file:
    annotations = json.load(json_file)

label_map = {
    0:'Enlarged Cardiomediastinum',
    1:'Cardiomegaly',
    2:'Lung Opacity',
    3:'Lung Lesion',
    4:'Edema',
    5:'Consolidation',
    6:'Pneumonia',
    7:'Atelectasis',
    8:'Pneumothorax',
    9:'Pleural Effusion',
    10:'Pleural Other',
    11:'Fracture',
    12:'Support Devices',
    13:'No Finding'
}

for split in annotations.keys():
    print(split, ':', len(annotations[split]))

train : 2069
val : 296
test : 590


In [3]:
reports_with_no_finding = []
condition_count = {}
for idx in range(14):
    condition_count[idx] = 0

for rep in list(labels.keys()):
    for idx in np.where(labels[rep] == 1)[0]:
        condition_count[idx] += 1
        if idx == 13:
            reports_with_no_finding.append(rep)

print('Distribution of various pseudo-labels across all splits:')
for condition in condition_count:
    print('\t', condition, ':', condition_count[condition])
    
print('Total number of reports with no finding:', len(reports_with_no_finding))

Distribution of various pseudo-labels across all splits:
	 0 : 39
	 1 : 193
	 2 : 270
	 3 : 80
	 4 : 65
	 5 : 22
	 6 : 98
	 7 : 188
	 8 : 15
	 9 : 99
	 10 : 22
	 11 : 39
	 12 : 30
	 13 : 2293
Total number of reports with no finding: 2293


In [4]:
train_reports = [item['id'] for item in annotations['train']]
print('Total number of reports in training data:', len(train_reports))

train_no_finding, train_others = [], []
for report in train_reports:
    if report in reports_with_no_finding:
        train_no_finding.append(report)
    else:
        train_others.append(report)

print('Number of reports in training data with no finding:', len(train_no_finding))

Total number of reports in training data: 2069
Number of reports in training data with no finding: 1517


In [5]:
downsampled_train_no_finding = random.choices(train_no_finding, k=150)
train_downsampled = train_others + downsampled_train_no_finding

train_annotations_downsampled = []

for report in annotations['train']:
    if report['id'] in train_downsampled:
        train_annotations_downsampled.append(report)

annotations['train'] = train_annotations_downsampled

In [6]:
for split in annotations.keys():
    print(split, ':', len(annotations[split]))

train : 693
val : 296
test : 590


In [7]:
with open('downsampled_annotations.json', 'w') as of:
    json.dump(annotations, of)